In [1]:
import os
import cv2
import numpy as np
from skimage.feature import local_binary_pattern
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.preprocessing import LabelEncoder
from tqdm import tqdm
import matplotlib.pyplot as plt
import kagglehub

path = kagglehub.dataset_download("ankit1743/skyview-an-aerial-landscape-dataset")
DATASET_DIR = os.path.join(path, 'Aerial_Landscapes')
GRID_SIZE = (4, 4)
LBP_P = 24
LBP_R = 3

def extract_spatial_lbp_features(image, grid_size=(4, 4)):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    h, w = gray.shape
    grid_h, grid_w = h // grid_size[0], w // grid_size[1]
    features = []

    for i in range(grid_size[0]):
        for j in range(grid_size[1]):
            block = gray[i*grid_h:(i+1)*grid_h, j*grid_w:(j+1)*grid_w]
            lbp = local_binary_pattern(block, P=LBP_P, R=LBP_R, method='uniform')
            hist, _ = np.histogram(lbp.ravel(),
                                   bins=np.arange(0, LBP_P + 3),
                                   range=(0, LBP_P + 2))
            hist = hist.astype("float")
            hist /= (hist.sum() + 1e-6)
            features.extend(hist)
    return features

def load_data(dataset_path):
    X, y = [], []
    classes = sorted(os.listdir(dataset_path))
    for class_name in classes:
        class_path = os.path.join(dataset_path, class_name)
        if not os.path.isdir(class_path):
            continue
        for img_name in tqdm(os.listdir(class_path), desc=f"Loading {class_name}"):
            img_path = os.path.join(class_path, img_name)
            img = cv2.imread(img_path)
            if img is None: continue
            img = cv2.resize(img, (128, 128))  # Normalize size
            features = extract_spatial_lbp_features(img, grid_size=GRID_SIZE)
            X.append(features)
            y.append(class_name)
    return np.array(X), np.array(y)

X, y = load_data(DATASET_DIR)
le = LabelEncoder()
y_encoded = le.fit_transform(y)
X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.2, random_state=42, stratify=y_encoded)


Loading River: 100%|██████████| 800/800 [00:14<00:00, 55.98it/s]


In [8]:
param_grid = {
    'C': [10, 100],
    'gamma': ['scale', 0.01],
    'kernel': ['poly']
}
grid = GridSearchCV(SVC(), param_grid, cv=5, verbose=2, n_jobs=-1)
grid.fit(X_train, y_train)

print("Best Parameters:", grid.best_params_)
best_model = grid.best_estimator_

y_pred = best_model.predict(X_test)
print(f"Accuracy: {accuracy_score(y_test, y_pred):.4f}")
print("Classification Report:")
print(classification_report(y_test, y_pred, target_names=le.classes_))
print("Confusion Matrix:")
print(confusion_matrix(y_test, y_pred))

Fitting 5 folds for each of 18 candidates, totalling 90 fits
Best Parameters: {'feature_selection__k': 300, 'svm__C': 10, 'svm__gamma': 'scale', 'svm__kernel': 'rbf'}
Training Accuracy: 0.5632
Test Accuracy: 0.5271
Difference: 0.0361


In [4]:
y_pred_train_svm = best_model.predict(X_train)
train_accuracy_svm = accuracy_score(y_train, y_pred_train_svm)
test_accuracy_svm = accuracy_score(y_test, y_pred)
print(f"SVM - Training accuracy: {train_accuracy_svm:.4f}, Test accuracy: {test_accuracy_svm:.4f}")
print(f"Difference: {train_accuracy_svm - test_accuracy_svm:.4f}")

SVM - Training accuracy: 0.7750, Test accuracy: 0.6362
Difference: 0.1388


In [5]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.decomposition import PCA

pca = PCA(n_components=100)
X_train_pca = pca.fit_transform(X_train)
X_test_pca = pca.transform(X_test)

param_grid_knn = {
    'n_neighbors': [3, 5, 7, 9],        
    'weights': ['uniform', 'distance'],
    'metric': ['euclidean', 'manhattan']
}

grid_knn = GridSearchCV(KNeighborsClassifier(), param_grid_knn, cv=5, verbose=2, n_jobs=-1)
grid_knn.fit(X_train_pca, y_train)

print("Best Parameters:", grid_knn.best_params_)
best_model_knn = grid_knn.best_estimator_

# Evaluation
y_pred = best_model_knn.predict(X_test_pca)
print(f"Accuracy: {accuracy_score(y_test, y_pred):.4f}")
print("Classification Report:")
print(classification_report(y_test, y_pred, target_names=le.classes_))
print("Confusion Matrix:")
print(confusion_matrix(y_test, y_pred))

Fitting 5 folds for each of 16 candidates, totalling 80 fits
Best Parameters: {'metric': 'manhattan', 'n_neighbors': 9, 'weights': 'distance'}
Accuracy: 0.3129
Classification Report:
              precision    recall  f1-score   support

 Agriculture       0.16      0.16      0.16       160
     Airport       0.29      0.19      0.23       160
       Beach       0.82      0.31      0.45       160
        City       0.23      0.32      0.26       160
      Desert       0.39      0.14      0.21       160
      Forest       0.31      0.76      0.44       160
   Grassland       0.61      0.64      0.63       160
     Highway       0.34      0.14      0.20       160
        Lake       0.55      0.38      0.45       160
    Mountain       0.20      0.43      0.28       160
     Parking       0.54      0.23      0.32       160
        Port       0.85      0.17      0.29       160
     Railway       0.45      0.14      0.21       160
 Residential       0.18      0.51      0.27       160
      

In [6]:
y_pred_train_knn = best_model_knn.predict(X_train_pca)
train_accuracy_knn = accuracy_score(y_train, y_pred_train_knn)
test_accuracy_knn = accuracy_score(y_test, y_pred)
print(f"SVM - Training accuracy: {train_accuracy_knn:.4f}, Test accuracy: {test_accuracy_knn:.4f}")
print(f"Difference: {train_accuracy_knn - test_accuracy_knn:.4f}")

SVM - Training accuracy: 1.0000, Test accuracy: 0.3129
Difference: 0.6871
[CV] END ......................C=10, gamma=scale, kernel=rbf; total time=  30.9s
[CV] END .......................C=10, gamma=0.01, kernel=rbf; total time=  40.3s
[CV] END .....................C=100, gamma=scale, kernel=rbf; total time=  24.7s
[CV] END .....................C=100, gamma=scale, kernel=rbf; total time=  22.3s
[CV] END ......................C=100, gamma=0.01, kernel=rbf; total time=  31.9s
[CV] END ...metric=euclidean, n_neighbors=3, weights=uniform; total time=   0.4s
[CV] END ..metric=euclidean, n_neighbors=3, weights=distance; total time=   0.2s
[CV] END ..metric=euclidean, n_neighbors=3, weights=distance; total time=   0.2s
[CV] END ...metric=euclidean, n_neighbors=5, weights=uniform; total time=   0.3s
[CV] END ..metric=euclidean, n_neighbors=5, weights=distance; total time=   0.2s
[CV] END ...metric=euclidean, n_neighbors=7, weights=uniform; total time=   0.3s
[CV] END ...metric=euclidean, n_nei